# Scraping notebook

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
from datetime import datetime

In [7]:
def get_fomc_minutes_links():
    base_url = "https://www.federalreserve.gov"
    archive_url = base_url + "/monetarypolicy/fomccalendars.htm"
    
    response = requests.get(archive_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    minutes_links = []
    for link in soup.find_all("a"):
        if "FOMC Minutes" in link.text:
            minutes_links.append(base_url + link["href"])
    
    return minutes_links

In [8]:
get_fomc_minutes_links()

[]

In [3]:
def get_minute_text_and_date(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.find('div', id='article').get_text(separator=' ', strip=True)
    date = soup.find('p', class_='article__time').get_text(strip=True)
    date = datetime.strptime(date, "Release Date: %B %d, %Y")
    return text, date

In [4]:
def scrape_fomc_minutes():
    links = get_minutes_links()
    minutes_data = []
    for link in links:
        text, date = get_minute_text_and_date(link)
        minutes_data.append({'date': date, 'text': text})
        time.sleep(1)  # To avoid overwhelming the server with requests
    return minutes_data

In [5]:
def save_to_csv(minutes_data):
    df = pd.DataFrame(minutes_data)
    df.to_csv('fomc_minutes.csv', index=False)

In [6]:
if __name__ == "__main__":
    minutes_data = scrape_fomc_minutes()
    save_to_csv(minutes_data)